In [1]:
import urllib.request
from bs4 import BeautifulSoup
import requests
import pandas as pd

## Здесь вытаскиваю координаты однушек {долгота, широта}

In [2]:
df = pd.read_csv("home")
df = df.drop(columns = "Unnamed: 0")
df.index.delete

<bound method Index.delete of RangeIndex(start=0, stop=2323, step=1)>

## Сначало отчищаю адресс от метро, загрузив список всех метро Москвы. Не со всеми метро это получилось, но и так сойдёт

In [5]:
f = open("metro")
metro = [i.replace("\n", "") for i in f]
f.close()
metro

['Авиамоторная',
 'Автозаводская',
 'Академическая',
 'Александровский сад',
 'Алексеевская',
 'Алма-Атинская',
 'Алтуфьево',
 'Аннино',
 'Арбатская (Арбатско-Покровская линия)',
 'Арбатская (Филевская линия)',
 'Аэропорт',
 'Бабушкинская',
 'Багратионовская',
 'Баррикадная',
 'Бауманская',
 'Беговая',
 'Белорусская',
 'Беляево',
 'Бибирево',
 'Библиотека имени Ленина',
 'Борисово',
 'Боровицкая',
 'Ботанический сад',
 'Братиславская',
 'Бульвар адмирала Ушакова',
 'Бульвар Дмитрия Донского',
 'Бульвар Рокоссовского',
 'Бунинская аллея',
 'Варшавская',
 'ВДНХ',
 'Владыкино',
 'Водный стадион',
 'Войковская',
 'Волгоградский проспект',
 'Волжская',
 'Волоколамская',
 'Воробьевы горы',
 'Выставочная',
 'Выхино',
 'Деловой центр',
 'Динамо',
 'Дмитровская',
 'Добрынинская',
 'Домодедовская',
 'Достоевская',
 'Дубровка',
 'Жулебино',
 'Зябликово',
 'Измайловская',
 'Калужская',
 'Кантемировская',
 'Каховская',
 'Каширская',
 'Киевская',
 'Китай-город',
 'Кожуховская',
 'Коломенская',
 'Ком

In [35]:
adr = []
for i in df["full_adr"]:
    adr.append(i.strip())

In [39]:
for i in range(len(adr)-1):
    for j in metro:
        adr[i] = adr[i].replace(j, "").strip()

In [95]:
coord = []
j = 0
for i in df["price"]:
    coord.append([adr[j], i])
    j+=1

In [96]:
coord

[['Москва ЮЗАО р-н Обручевский Ленинский просп. 109/1к1', 32900],
 ['Москва ЮАО р-н  ул. Мусы Джалиля 38К1', 33000],
 ['Москва СЗАО р-н Щукино Новощукинская ул. 2  Стрешнево', 33000],
 ['Москва ЗАО р-н -Давыдково Кастанаевская ул. 35К1', 33000],
 ['Москва СВАО р-н  Мурановская ул. 13', 33000],
 ['Москва СЗАО р-н  Митинская ул. 31', 33000],
 ['Москва НАО (Новомосковский) Сосенское поселение ул. Александры Монаховой 109к2',
  33000],
 ['Москва ЮВАО р-н  Белореченская ул. 34К1', 33000],
 ['Москва ВАО р-н Северное Измайлово 13-я Парковая ул. 40', 33000],
 ['Москва ЮАО р-н  Севанская ул. 5К1', 33000],
 ['Москва СВАО р-н  Северный бул. 3К2', 33000],
 ['Москва САО р-н Ховрино Клинская ул. 18к2 Ховрино', 33000],
 ['Москва ЮВАО р-н - Жулебинский бул. 2К2', 33000],
 ['Москва НАО (Новомосковский) Московский мкр. Град Московский ул. Радужная 25 Саларьево',
  27000],
 ['Москва ЮЗАО р-н Южное Бутово Южнобутовская ул. 80К1  Бульвар Адмирала Ушакова',
  25000],
 ['Москва ЦАО р-н Пресненский Ходынская 

## Смотрю какой ответ получается после get запроса к яндекс API 

In [92]:
l = "Москва ВАО р-н ьники Маленковская ул. 12 ьники".replace(" ", "+")

In [94]:
k = requests.get("https://geocode-maps.yandex.ru/1.x/?format=json&geocode="+l).json()
k

{'response': {'GeoObjectCollection': {'featureMember': [{'GeoObject': {'Point': {'pos': '37.775631 55.78771'},
      'boundedBy': {'Envelope': {'lowerCorner': '37.648133 55.695281',
        'upperCorner': '37.945664 56.009688'}},
      'description': 'Москва, Россия',
      'metaDataProperty': {'GeocoderMetaData': {'Address': {'Components': [{'kind': 'country',
           'name': 'Россия'},
          {'kind': 'province', 'name': 'Центральный федеральный округ'},
          {'kind': 'province', 'name': 'Москва'},
          {'kind': 'locality', 'name': 'Москва'},
          {'kind': 'district', 'name': 'Восточный административный округ'}],
         'country_code': 'RU',
         'formatted': 'Москва, Восточный административный округ'},
        'AddressDetails': {'Country': {'AddressLine': 'Москва, Восточный административный округ',
          'AdministrativeArea': {'AdministrativeAreaName': 'Москва',
           'Locality': {'DependentLocality': {'DependentLocalityName': 'Восточный администр

In [89]:
e = k['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']['Point']['pos'].split()
e.reverse()
e

['55.825153', '37.595887']

## Итерируясь по адресам, нахожу координаты, записываю их в массив. try блок и j для сохранения результата, в случае ошибки

In [103]:
j=2298
try:
    for i in coord[j:]:
        k = requests.get("https://geocode-maps.yandex.ru/1.x/?format=json&geocode="+i[0]).json()
        if k['response']['GeoObjectCollection']['featureMember']:
            e = k['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']['Point']['pos'].split()
            e.reverse()
            coord[j].append(e)
            j+=1
except Exception:
    print(j)

2299


In [101]:
coord

[['Москва ЮЗАО р-н Обручевский Ленинский просп. 109/1к1',
  32900,
  ['55.664106', '37.513601']],
 ['Москва ЮАО р-н  ул. Мусы Джалиля 38К1', 33000, ['55.617027', '37.74286']],
 ['Москва СЗАО р-н Щукино Новощукинская ул. 2  Стрешнево',
  33000,
  ['55.808882', '37.459172']],
 ['Москва ЗАО р-н -Давыдково Кастанаевская ул. 35К1',
  33000,
  ['55.728003', '37.443533']],
 ['Москва СВАО р-н  Мурановская ул. 13', 33000, ['55.894202', '37.599777']],
 ['Москва СЗАО р-н  Митинская ул. 31', 33000, ['55.846805', '37.356405']],
 ['Москва НАО (Новомосковский) Сосенское поселение ул. Александры Монаховой 109к2',
  33000,
  ['55.541739', '37.497899']],
 ['Москва ЮВАО р-н  Белореченская ул. 34К1',
  33000,
  ['55.661262', '37.774445']],
 ['Москва ВАО р-н Северное Измайлово 13-я Парковая ул. 40',
  33000,
  ['55.808169', '37.812965']],
 ['Москва ЮАО р-н  Севанская ул. 5К1', 33000, ['55.620103', '37.662785']],
 ['Москва СВАО р-н  Северный бул. 3К2', 33000, ['55.865581', '37.605885']],
 ['Москва САО р-н Х

In [98]:
j

2285

In [108]:
full_cor = []
for i in coord:
    if len(i)==3:
        full_cor.append(i)

In [110]:
df2 = pd.DataFrame(full_cor)

In [112]:
df2.to_csv("full_corr")

In [122]:
f = open("ful_data", 'w')

## Записываю массив в файл с координатами и ценой, чтобы можно было его явно вставить в javascript, лучшего решения я не нашёл.

In [123]:
f.write("[")
for i in full_cor:    
    f.write("[" + str(i[2]) + "," + str(i[1]) +"]"+",")
f.write("]")

1

In [124]:
f.close()

In [9]:
df.describe()

,price
count,2323.000000
mean,32901.314679
std,24832.568045
min,10000.000000
25%,24000.000000
50%,27000.000000
75%,32000.000000
max,399000.000000
